In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import torch
import math
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import time
import os
import copy

import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import torch


In [ ]:
def load_text(path,category):
  with open(path, 'rb') as f:
    texts = []
    labels2 = []
    for line in f:
      texts.append(line.decode(errors='ignore').lower().strip())
      labels2.append(category)

  return texts, labels2
neg_text, neg_labels = load_text('/content/drive/MyDrive/Bengali-Sentiment/all_negative_3307.txt', category=0)
pos_text, pos_labels = load_text('/content/drive/MyDrive/Bengali-Sentiment/all_positive_8500.txt', category=1)

texts = np.array(neg_text + pos_text)
labels = np.array(neg_labels+pos_labels)
#labels = np.array([0]*len(neg_text) + [1]*len(pos_text))

In [ ]:
def preprocess(text):
    text = str(text).replace('।', '\n')
    whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
    bangla_fullstop = u"\u0964"
    punctSeq = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
    punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
    text = whitespace.sub(" ", text).strip()
    text = re.sub(punctSeq, " ", text)
    text = re.sub(punc, " ", text)
    text = "".join(i for i in text if ord(i) > ord('z') or ord(i) == 32)
    text = re.sub(' +', ' ', text)
    return (text)


In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
  max_len = 0
  tokenized_texts = []
  word2idx ={} 

  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1
  idx = 2
  for sent in texts:
    tokenized_sent = sent.split()
    tokenized_texts.append(tokenized_sent)
    for token in tokenized_sent:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1
    max_len = max(max_len, len(tokenized_sent))
  return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
  input_ids = []
  for tokenized_sent in tokenized_texts:
    tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))
    input_id = [word2idx.get(token) for token in tokenized_sent]
    input_ids.append(input_id)
  return np.array(input_ids)

In [ ]:

def load_glove(word2idx, filenameg, vector_size):
  embedding_vectors = np.random.uniform(-0.25, 0.25, (len(word2idx), vector_size))
  f = open(filenameg,encoding='utf-8', errors='ignore')
  embedding_vectors[word2idx['<pad>']] = np.zeros((vector_size,))
  count = 0
  for line in f:
    values = line.split()
    word = values[0]
    #print(values[1:])
    vectorg = np.asarray(values[1:], dtype="float32")
    if word in word2idx:
      count += 1
      embedding_vectors[word2idx[word]] = vectorg
      #print(f"There are {count} / {len(word2idx)} pretrained vectors found.")
  f.close()
  return embedding_vectors

In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,batch_size=50):
  train_inputs, val_inputs, train_labels, val_labels =\
  tuple(torch.tensor(data) for data in[train_inputs, val_inputs, train_labels, val_labels])
  batch_size = 50
  train_data = TensorDataset(train_inputs, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  val_data = TensorDataset(val_inputs, val_labels)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
  return train_dataloader, val_dataloader

In [ ]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

print(len(word2idx))
embedding_dim = 200
# Load pretrained vectors
embeddings = load_glove(word2idx, '/content/drive/MyDrive/Bengali-Sentiembedding/GloVe-300.txt',200)
embeddings = torch.tensor(embeddings)

Tokenizing...

24250


In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1, random_state=42)

train_dataloader, val_dataloader =\
data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [ ]:
class CNN_NLP(nn.Module):
  def __init__(self):
    super(CNN_NLP, self).__init__()
    pretrained_embedding=embeddings
    freeze_embedding=False
    vocab_size=len(word2idx)
    embed_dim=200
    filter_sizes=[3, 4, 5]
    num_filters=[100, 100, 100]
    num_classes=2
    dropout=0.5
    self.num_layers = 2
    self.hidden_dim=256
    self.vocab_size, self.embed_dim = pretrained_embedding.shape
    self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,freeze=freeze_embedding)
    self.conv1 =  nn.Conv1d(in_channels=self.embed_dim,out_channels=100,kernel_size=3)
    self.conv2 =  nn.Conv1d(in_channels=self.embed_dim,out_channels=100,kernel_size=4)
    self.conv3 =  nn.Conv1d(in_channels=self.embed_dim,out_channels=100,kernel_size=5)
    self.lstm = nn.LSTM(300,256,self.num_layers,bidirectional=True,batch_first=True)
    self.fc = nn.Linear(512, num_classes)
    #self.fc = nn.Linear(np.sum(num_filters), num_classes)
    self.dropout = nn.Dropout(p=dropout)
  def forward(self, input_ids):
    x_embed = self.embedding(input_ids).float()
    x_reshaped = x_embed.permute(0, 2, 1)
    x1 = F.relu(self.conv1(x_reshaped))
    x2 = F.relu(self.conv2(x_reshaped))
    x3 = F.relu(self.conv3(x_reshaped))
    x1 = F.max_pool1d(x1,kernel_size=x1.shape[2])
    x2 = F.max_pool1d(x2,kernel_size=x2.shape[2])
    x3 = F.max_pool1d(x3,kernel_size=x3.shape[2])
    fc_x = torch.cat([x1.squeeze(dim=2),x2.squeeze(dim=2),x3.squeeze(dim=2)], dim=1)
    batch_size = input_ids.shape[0]  
    h = torch.zeros((self.num_layers*2, self.hidden_dim))
    c = torch.zeros((self.num_layers*2, self.hidden_dim))
    torch.nn.init.xavier_normal_(h)
    torch.nn.init.xavier_normal_(c)
    h = h.to(device)
    c = c.to(device)
    out, (hidden, cell) = self.lstm(fc_x, (h,c))
    out =  F.relu(out)
    out = self.dropout(out)
    #print(out)
    out = self.fc(out)
    return out

In [ ]:
import torch.optim as optim

def initilize_model():
  cnn_model = CNN_NLP()
  cnn_model.to(device)
  optimizer = optim.Adadelta(cnn_model.parameters(),lr=0.25,rho=0.95)
  return cnn_model, optimizer

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
  best_accuracy = 0
  print("Start training...\n")
  print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
  print("-"*60)
  best_model = copy.deepcopy( model.state_dict() )
  for epoch_i in range(epochs):
    t0_epoch = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
      b_input_ids, b_labels = tuple(t.to(device) for t in batch)
      model.zero_grad()
      logits = model(b_input_ids)
      loss = loss_fn(logits, b_labels)
      total_loss += loss.item()
      loss.backward()
      optimizer.step()
      avg_train_loss = total_loss / len(train_dataloader)

    if val_dataloader is not None:
      val_loss, val_accuracy = evaluate(model, val_dataloader)
      #print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}") 
      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = copy.deepcopy( model.state_dict() )
        time_elapsed = time.time() - t0_epoch
        print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")        
  print("\n")
  print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")
  return best_model

def evaluate(model, val_dataloader):
  model.eval()
  val_accuracy = []
  val_loss = []
  for batch in val_dataloader:
    b_input_ids, b_labels = tuple(t.to(device) for t in batch)
    with torch.no_grad():
      logits = model(b_input_ids)
      loss = loss_fn(logits, b_labels)
      val_loss.append(loss.item())
      preds = torch.argmax(logits, dim=1).flatten()
      accuracy = (preds == b_labels).cpu().numpy().mean() * 100
      val_accuracy.append(accuracy)
  val_loss = np.mean(val_loss)
  val_accuracy = np.mean(val_accuracy)
  return val_loss, val_accuracy

In [ ]:
set_seed(42)
cnn_non_static, optimizer = initilize_model()
best_model = train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=50)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   0.583357   |  0.531831  |   71.59   |   4.57   
   2    |   0.431690   |  0.325480  |   85.30   |   4.71   
   3    |   0.301634   |  0.255597  |   89.26   |   4.66   
   4    |   0.246326   |  0.235336  |   89.76   |   4.54   
   5    |   0.214474   |  0.218859  |   91.78   |   5.50   
   9    |   0.111365   |  0.206079  |   92.12   |   4.58   
  10    |   0.085604   |  0.212796  |   92.28   |   4.60   
  15    |   0.024495   |  0.265907  |   92.62   |   4.51   
  17    |   0.017046   |  0.266146  |   92.65   |   4.73   
  18    |   0.016812   |  0.282680  |   92.81   |   4.60   
  19    |   0.014582   |  0.287448  |   93.03   |   5.04   
  29    |   0.003086   |  0.327794  |   93.20   |   4.52   
  41    |   0.003484   |  0.343565  |   93.28   |   4.51   


Training complete! Best accuracy: 93.28%.
